# 1 Dependency import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import set_config

from xgboost import XGBRegressor

In [23]:
def evaluate(model, grid_params):
    set_config(display="diagram")
    # set_config(display="text")

    categorical_transformer_pipeline = Pipeline(steps=[
        ("cat_imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    column_transformer = ColumnTransformer([
        ("num_imputer", SimpleImputer(strategy="mean"), numerical_cols),
        ("cat_transformer_pipeline", categorical_transformer_pipeline, categorical_cols)
    ])

    pipeline = Pipeline([
        ("transforms", column_transformer),
        ("model", DecisionTreeRegressor())
    ])

    gs = GridSearchCV(pipeline, grid_params, scoring="neg_root_mean_squared_error").fit(X_train, y_train)
    
    score = gs.score(X_test, y_test)

    return score, gs.best_params_, pipeline
    
    # print(f"MSE: {score}\nBest paramas: {gs.best_params_}")

In [4]:
# def evaluate(model):
#     numerical_transformer = SimpleImputer(strategy="mean")

#     categorical_transformer = Pipeline(steps=[
#         ("imputer", SimpleImputer(strategy="most_frequent")),
#         ("onehot", OneHotEncoder(handle_unknown="ignore"))
#     ])

#     preprocessor = ColumnTransformer(
#         transformers=[
#             ("num", numerical_transformer, numerical_cols),
#             ("cat", categorical_transformer, categorical_cols)
#         ]
#     )

#     pipeline = Pipeline(steps=[
#         ("preprocessor", preprocessor),
#         ("model", model)
#     ])

#     mae = -1 * cross_val_score(pipeline, X, y, cv=5, scoring="neg_mean_absolute_error").mean()
#     print(f"MAE: {mae}")

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

***
# 2 Loading data

In [2]:
data = pd.read_csv("data/2015-cleaned.csv", delimiter=",")

In [3]:
#target = "GHGEmissionsIntensity(kgCO2e/ft2)"
target = "SiteEnergyUse(kBtu)"

In [4]:
y = data[target]
X = data.drop(columns=[target])

In [5]:
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [6]:
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [7]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [8]:
y_test.describe()

count    6.660000e+02
mean     4.606876e+06
std      1.055033e+07
min      0.000000e+00
25%      8.487488e+05
50%      1.619646e+06
75%      3.423317e+06
max      1.362414e+08
Name: SiteEnergyUse(kBtu), dtype: float64

***
# 3 Feature engineering

## 3.1 Mutual Information

In [ ]:
X_MI = X.copy()

In [ ]:
for col in X_MI.select_dtypes("object"):
    X_MI[col], _ = X_MI[col].factorize()

In [ ]:
discrete_features = X_MI.dtypes == int

In [ ]:
# make_mi_scores(X_MI, y, discrete_features)

***
# 4 Modeling

***
## 4.1 DummyRegressor

### 4.1.1 Strategy 1

In [ ]:
%%time

model = DummyRegressor(strategy="mean")
evaluate(model)

***
### 4.1.2 Strategy 2

In [ ]:
%%time

model = DummyRegressor(strategy="median")
evaluate(model)

***
### 4.1.3 Strategy 3

In [ ]:
%%time

for q in range(0, 10, 1):
    model = DummyRegressor(strategy="quantile", quantile=q/10)
    evaluate(model)

***
### 4.1.4 Strategy 4

In [ ]:
%%time

model = DummyRegressor()

grid_params = {
    "model__strategy": ["mean", "median"]
}

evaluate(model)

***
## 4.2 DecisionTreeRegressor

### 4.2.1 Strategy 1

In [ ]:
%%time

model = DecisionTreeRegressor(random_state=0)
evaluate(model, X_train, y_train)

***
### 4.2.2 Strategy 2

In [ ]:
%%time

for md in range(1, 21, 1):
    model = DecisionTreeRegressor(max_depth=md, random_state=0)
    evaluate(model, X_train, y_train)

***
### 4.2.3 Strategy 3

In [ ]:
%%time

for msl in range(1, 10, 1):
    model = DecisionTreeRegressor(min_samples_leaf=msl, random_state=0)
    evaluate(model)

***
### 4.2.4 Strategy 4

In [ ]:
%%time

for s in ["mse", "friedman_mse", "mae", "poisson"]:
    model = DecisionTreeRegressor(criterion=s, random_state=0)
    evaluate(model)

***
### 4.2.5 Strategy 5

In [ ]:
%%time

model = DecisionTreeRegressor(min_samples_leaf=3, criterion="mae", random_state=0)
evaluate(model)

***
## 4.3 RandomForestRegressor

### 4.2.1 Strategy 1

In [ ]:
%%time

model = RandomForestRegressor(random_state=1)
evaluate(model)

***
### 4.2.2 Strategy 2

In [ ]:
%%time

for ne in range(50, 250, 50):
    RandomForestRegressor(n_estimators=ne, random_state=1)
    evaluate(model)

***
### 4.2.3 Strategy 3

In [ ]:
%%time

for c in ["mse", "mae", "poisson"]:
    RandomForestRegressor(criterion=c, random_state=1)
    evaluate(model)

***
### 4.2.4 Strategy 4

In [ ]:
%%time

for msl in range(5, 25, 5):
    RandomForestRegressor(min_samples_leaf=msl, random_state=1)
    evaluate(model)

***
## 4.4 XGBRegressor

### 4.4.1 Strategy 1

In [ ]:
%%time

model = XGBRegressor(random_state=1)
evaluate(model)

***
### 4.4.2 Strategy 2

In [ ]:
%%time

for ne in [100, 200, 500, 1000]:
    model = XGBRegressor(n_estimators=ne, random_state=1)
    evaluate(model)

***
### 4.4.3 Strategy 3

In [ ]:
%%time

for lr in [0.05, 0.075, 0.09, 0.1, 0.11, 0.12, 0.15]:
    model = XGBRegressor(learning_rate=lr, n_estimators=200, random_state=1, verbosity=0)
    print(f"learning_rate:  {lr}")
    evaluate(model)

***
### 4.4.3 Strategy 3

In [ ]:
%%time

for md in [2, 3, 4, 5]:
    model = XGBRegressor(max_depth=md, learning_rate=0.09, n_estimators=200, random_state=1, verbosity=0)
    print(f"max_depth:  {md}")
    evaluate(model)

***
# 5 ...

In [9]:
set_config(display="diagram")

categorical_transformer_pipeline = Pipeline(steps=[
    ("cat_imputer", SimpleImputer(strategy="most_frequent")),
    ("cat_encoder", OneHotEncoder(handle_unknown="ignore"))
])

column_transformer = ColumnTransformer([
    ("num_imputer", SimpleImputer(strategy="mean"), numerical_cols),
    ("cat_transformer_pipeline", categorical_transformer_pipeline, categorical_cols)
])

pipeline = Pipeline([
    ("transforms", column_transformer),
    ("model", DecisionTreeRegressor())
])

In [10]:
display(pipeline)

Pipeline(steps=[('transforms',
                 ColumnTransformer(transformers=[('num_imputer',
                                                  SimpleImputer(),
                                                  ['Unnamed: 0',
                                                   'OSEBuildingID', 'DataYear',
                                                   'CouncilDistrictCode',
                                                   'YearBuilt',
                                                   'NumberofBuildings',
                                                   'NumberofFloors',
                                                   'PropertyGFATotal',
                                                   'PropertyGFAParking',
                                                   'PropertyGFABuilding(s)',
                                                   'LargestPropertyUseTypeGFA',
                                                   'SecondLargestPropertyUseTypeGFA',
                                                   'ThirdLargestProper...
                                                   'Seattle Police Department '
                                                   'Micro Community Policing '
                                                   'Plan Areas',
                                                   'City Council Districts', ...]),
                                                 ('cat_transformer_pipeline',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['BuildingType',
                                                   'DefaultData', 'Comment',
                                                   'ComplianceStatus',
                                                   'Outlier'])])),
                ('model', DecisionTreeRegressor())])

In [20]:
# pipeline.get_params()

In [16]:
grid_params = {
        "model__random_state": [1],
        "model__max_depth": range(2, 22, 2),
        "model__min_samples_leaf": range(1, 11, 1),
        "model__criterion": ["mse"]
    }

In [17]:
gs = GridSearchCV(pipeline, grid_params, scoring="neg_root_mean_squared_error").fit(X_train, y_train)

In [18]:
set_config(display="text")

gs.score(X_test, y_test)

-875568.2279267329

In [19]:
gs.best_params_

{'model__criterion': 'mse',
 'model__max_depth': 10,
 'model__min_samples_leaf': 1,
 'model__random_state': 1}